In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# IMPORTS

In [14]:
import os
from dotenv import load_dotenv

# data
import pandas as pd

# gensim
import gensim
from gensim.corpora.dictionary import Dictionary

# spacy
from spacy.lang.de.stop_words import STOP_WORDS
STOP_WORDS = list(STOP_WORDS)

# scikit
from sklearn.feature_extraction.text import TfidfVectorizer # ,CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
# from sklearn.datasets import fetch_20newsgroups

# topicwizard
import topicwizard
from topicwizard.pipeline import make_topic_pipeline
# from topicwizard.compatibility import gensim_pipeline

# pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models

# src
from src.general.io import to_pickle, from_pickle
from src.general.io import disable_warnings
from src.models.custom_lda import CustomLDA

In [15]:
disable_warnings()

# PATHS & NAMES

In [16]:
load_dotenv()

RAW_DATA_PATHS = {}
for short_name, filename in zip(["feb", "mar", "apr", "may", "feedback"],
                                ["event_stg_user_input_web_cw6.txt",
                                "event_stg_user_input_web_cw10.txt",
                                "event_stg_user_input_web_cw15.txt",
                                "event_stg_user_input_web_cw19.txt",
                                "current_user_feedback_text.xlsx",
                                ]):
    RAW_DATA_PATHS[short_name] = f"..{os.getenv('RAW_DATA_DIR')}/{filename}"


EXT_DATA_PATHS = {}
for short_name, filename in zip(["rasa"],
                                ["rasa_train_data.pkl"]):
    EXT_DATA_PATHS[short_name] = f"..{os.getenv('EXTERNAL_DATA_DIR')}/{filename}"


INTERIM_DATA_PATHS = {}
for short_name, filename in zip(["rasa_docs", "rasa_emb"],
                                ["rasa_docs.pkl",
                                "rasa_embedings.pkl",
                                ]):
    INTERIM_DATA_PATHS[short_name] = f"..{os.getenv('INTERIM_DATA_DIR')}/{filename}"

# add more interims
for short_name in RAW_DATA_PATHS.keys():
    INTERIM_DATA_PATHS[f"{short_name}_doc"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_docs.pkl"
    INTERIM_DATA_PATHS[f"{short_name}_emb"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_embeding.pkl"
    INTERIM_DATA_PATHS[f"{short_name}_tok"] = f"..{os.getenv('INTERIM_DATA_DIR')}/{short_name}_tokens.pkl"


EDA_REPORT_PATHS = {}
for short_name in RAW_DATA_PATHS.keys():
    EDA_REPORT_PATHS[short_name] = f"..{os.getenv('REPORT_DIR')}/eda_{short_name}.html"

                        
TEXT_COL_USERINPUT = "user_input"
ID_COL_USERINPUT = "session_id"

N_TOPICS = 20
MONTHS = ["feb", "mar", "apr", "may"]

In [17]:
# RAWFILE_FOLDER = "../data/raw"
# INTERIM_FOLDER = "../data/interim"
# EXTERNAL_FOLDER = "../data/external"
# REPORT_FOLDER = "../reports"
# MODELS_FOLDER = "../models"

# # processed
# FEEDBACK_PROCESSED_FILEPATH = f"{INTERIM_FOLDER}/feedback_docs.pkl"
# RASA_DOCS_FILEPATH = f"{INTERIM_FOLDER}/rasa_docs.pkl"
# RASA_EMBEDINGS_FILEPATH = f"{INTERIM_FOLDER}/rasa_embedings.pkl"
# RASA_FILEPATH = f"{EXTERNAL_FOLDER}/rasa_train_data.pkl"

# FEB_DOCS_FILEPATH = f"{INTERIM_FOLDER}/february_docs.pkl"
# MAR_DOCS_FILEPATH = f"{INTERIM_FOLDER}/march_docs.pkl"
# APR_DOCS_FILEPATH = f"{INTERIM_FOLDER}/april_docs.pkl"
# MAY_DOCS_FILEPATH = f"{INTERIM_FOLDER}/may_docs.pkl"

# FEB_TOKENS_FILEPATH = f"{INTERIM_FOLDER}/february_tokens.pkl"
# MAR_TOKENS_FILEPATH = f"{INTERIM_FOLDER}/march_tokens.pkl"
# APR_TOKENS_FILEPATH = f"{INTERIM_FOLDER}/april_tokens.pkl"
# MAY_TOKENS_FILEPATH = f"{INTERIM_FOLDER}/may_tokens.pkl"

# FEB_EMBEDING_FILEPATH = f"{INTERIM_FOLDER}/february_embeding.pkl"
# MAR_EMBEDING_FILEPATH = f"{INTERIM_FOLDER}/march_embeding.pkl"
# APR_EMBEDING_FILEPATH = f"{INTERIM_FOLDER}/april_embeding.pkl"
# MAY_EMBEDING_FILEPATH = f"{INTERIM_FOLDER}/may_embeding.pkl"

# CUSTOM_LDA_PATH = f"{MODELS_FOLDER}/lda.pkl"

# SEED = 42
# N_TOPICS = 20

# DATA

In [18]:
token_data = {}
docs_data = {}
for month in MONTHS:
    token_data[month] = from_pickle(INTERIM_DATA_PATHS[f"{month}_tok"])
    docs_data[month] = from_pickle(INTERIM_DATA_PATHS[f"{month}_doc"])

# Topic modelling

## gensim

### LDA

In [19]:
# # Fit LDA model
# gensim_model = gensim.models.ldamodel.LdaModel(
#     corpus = corpus,      # Document-Term Matrix
#     id2word = id2word,    # Map word IDs to words
#     num_topics = N_TOPICS,      # Number of latent topics to extract
#     random_state = SEED,
#     passes = 100,         # N° of passes through the corpus during training
#     )

In [20]:
# # Visualize with pyLDAvis
# pyLDAvis.enable_notebook()
# visualization = pyLDAvis.gensim_models.prepare(gensim_model, corpus, id2word, mds = "mmds", R = 21)
# visualization

### Custom LDA

In [21]:
custom_lda = CustomLDA(token_data["feb"], seed=int(os.getenv('SEED')))

#### hyper-parameters optimisation

In [22]:
if 1==2:
    custom_lda.grid_search_hyperparameters()
    to_pickle(custom_lda, f"{os.getenv('MODELS_DIR')}/lda.pkl")

    print(f"{custom_lda.best_num_topics=}, {custom_lda.best_alpha=}, {custom_lda.best_beta=}")
    custom_lda.df_grid_results.head()

#### 50 topics

##### Febraury

In [24]:
custom_lda.fit() # when no parameters are provied best (estimated by grid search) would be used
custom_lda.visualize("train")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.469825  0.128803       1        1  10.048836
26     0.239580  0.328140       2        1   7.105226
23     0.191207  0.350078       3        1   5.911692
28    -0.395187  0.075033       4        1   4.857024
16    -0.353784 -0.187183       5        1   4.789701
27    -0.307871  0.207202       6        1   4.579601
21    -0.021006  0.271483       7        1   3.983832
22    -0.155018  0.112612       8        1   3.899086
17    -0.357997 -0.314604       9        1   3.763277
2     -0.304878  0.064008      10        1   3.486493
11     0.253322  0.239773      11        1   3.414877
8      0.012557  0.391977      12        1   3.220597
18    -0.232755  0.307614      13        1   3.137087
7     -0.345844 -0.045404      14        1   3.004677
29    -0.035093 -0.039427      15        1   2.868975
1     -0.124042  0.425123      16        1   2.613333
14     0.411124 -0.022439      17        1   2.527786
6     -0.025479  0.150177      18        1   2.496321
25     0.085857  0.071891      19        1   2.357804
20     0.266797  0.071467      20        1   2.356640
13    -0.235701 -0.135449      21        1   2.286997
4      0.202129 -0.052739      22        1   2.232283
5      0.380564 -0.207406      23        1   2.203088
24    -0.205569 -0.329423      24        1   2.097790
3     -0.069964 -0.190110      25        1   2.063355
15     0.047964 -0.200320      26        1   2.038355
0      0.308895 -0.319653      27        1   1.822790
19     0.006024 -0.357980      28        1   1.668613
12     0.226648 -0.344039      29        1   1.591464
10     0.067694 -0.449203      30        1   1.572400, topic_info=               Term         Freq        Total Category  logprob  loglift
1              kein  1336.000000  1336.000000  Default  21.0000  21.0000
14          storung   746.000000   746.000000  Default  20.0000  20.0000
7          internet  1510.000000  1510.000000  Default  19.0000  19.0000
16             wlan   378.000000   378.000000  Default  18.0000  18.0000
3     funktionieren   503.000000   503.000000  Default  17.0000  17.0000
...             ...          ...          ...      ...      ...      ...
1331  verfugbarkeit     0.758764     1.718277  Topic30  -6.2654   3.3352
1113      standiges     0.758764     1.718277  Topic30  -6.2654   3.3352
603          hierzu     0.758654     1.718167  Topic30  -6.2656   3.3351
641         laufend     2.289791    13.449938  Topic30  -5.1609   2.3821
1307           alle     0.758797     2.368052  Topic30  -6.2654   3.0145

[899 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
198       4  0.836527         ab
1062     18  0.942058  abbrechen
575      28  0.945314    abbruch
122      27  0.919568   abbruche
163       8  0.860376      abend
...     ...       ...        ...
534      28  0.904129     zuruck
338       7  0.955269   zusammen
196       5  0.093266       zwar
196       6  0.373062       zwar
196      28  0.466328       zwar

[1024 rows x 3 columns], R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 27, 24, 29, 17, 28, 22, 23, 18, 3, 12, 9, 19, 8, 30, 2, 15, 7, 26, 21, 14, 5, 6, 25, 4, 16, 1, 20, 13, 11])

##### March

In [26]:
custom_lda.add_corpus(token_data["mar"], "mar")
custom_lda.visualize("mar")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27    -0.281535  0.066721       1        1  14.933477
28    -0.136626  0.247562       2        1   7.948676
9      0.407634 -0.265055       3        1   7.545508
8      0.272054  0.268833       4        1   4.007196
2     -0.163500  0.126322       5        1   3.944585
18    -0.075706 -0.290933       6        1   3.780913
16    -0.302962  0.249754       7        1   3.707012
29     0.169472  0.111320       8        1   3.592771
17    -0.427362  0.205485       9        1   3.433711
13    -0.277371 -0.118845      10        1   3.388298
11     0.336346  0.050961      11        1   3.328564
22     0.176733 -0.160261      12        1   3.327585
23     0.368845 -0.079158      13        1   3.196730
26     0.398828 -0.022039      14        1   3.167764
7     -0.115237  0.330412      15        1   2.883252
14     0.248121 -0.337149      16        1   2.592104
19     0.098712 -0.366546      17        1   2.416229
25    -0.100361 -0.039064      18        1   2.374815
6      0.074840  0.039676      19        1   2.369640
21     0.138021  0.382391      20        1   2.307839
15     0.011361  0.340708      21        1   2.142393
20     0.136408  0.251114      22        1   2.035765
24    -0.251388  0.328447      23        1   1.727422
0     -0.059013 -0.455807      24        1   1.586258
1      0.390001  0.198362      25        1   1.490653
5     -0.250398 -0.376319      26        1   1.434129
3      0.046759 -0.103845      27        1   1.371297
4     -0.057917 -0.191098      28        1   1.343169
10    -0.358567 -0.263362      29        1   1.340002
12    -0.416193 -0.128586      30        1   1.282243, topic_info=                 Term        Freq       Total Category  logprob  loglift
1                kein  201.000000  201.000000  Default  21.0000  21.0000
14            storung  126.000000  126.000000  Default  20.0000  20.0000
7            internet  213.000000  213.000000  Default  19.0000  19.0000
134         techniker  139.000000  139.000000  Default  18.0000  18.0000
5               nicht  209.000000  209.000000  Default  17.0000  17.0000
...               ...         ...         ...      ...      ...      ...
1275          browser    0.125097    0.450782  Topic30  -6.2258   3.0747
154      unterbrechen    1.208420    9.277145  Topic30  -3.9578   2.3183
1                kein   14.776977  201.895230  Topic30  -1.4541   1.7419
392   leitungsproblem    0.004846    0.194871  Topic30  -9.4767   0.6624
0                 dsl    0.004846   29.863744  Topic30  -9.4767  -4.3697

[902 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
198       2  0.714459         ab
1062     19  1.015354  abbrechen
575      17  1.037310    abbruch
122      24  0.827259   abbruche
163      12  0.636052      abend
...     ...       ...        ...
161       2  0.933549    zuhause
534      17  0.831523     zuruck
338      20  0.619179   zusammen
196       1  0.694242       zwar
196      17  0.231414       zwar

[836 rows x 3 columns], R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 29, 10, 9, 3, 19, 17, 30, 18, 14, 12, 23, 24, 27, 8, 15, 20, 26, 7, 22, 16, 21, 25, 1, 2, 6, 4, 5, 11, 13])

In [ ]:
# # new corpus
# corpus_mar = [custom_lda.id2word.doc2bow(doc) for doc in tokens_mar]
# predictions_apr = custom_lda.model[corpus_mar]

# # visual
# pyLDAvis.enable_notebook()
# visualization = pyLDAvis.gensim_models.prepare(custom_lda.model, corpus_apr, custom_lda.id2word, mds="mmds", R=21)
# visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.122086 -0.295445       1        1  10.561008
32     0.141218  0.312399       2        1   6.004005
37    -0.336073  0.041759       3        1   3.841017
38     0.032688  0.056192       4        1   1.936763
42    -0.021641  0.026711       5        1   1.864808
9     -0.002380  0.008810       6        1   1.829266
48     0.001458 -0.003419       7        1   1.680990
22     0.001458 -0.003419       8        1   1.680990
25     0.001459 -0.003418       9        1   1.680989
39     0.001459 -0.003418      10        1   1.680988
31     0.001458 -0.003419      11        1   1.680988
18     0.001459 -0.003418      12        1   1.680987
33     0.001456 -0.003419      13        1   1.680987
49     0.001455 -0.003419      14        1   1.680986
13     0.001457 -0.003419      15        1   1.680986
3      0.001457 -0.003419      16        1   1.680985
30     0.001455 -0.003419      17        1   1.680985
28     0.001457 -0.003419      18        1   1.680985
12     0.001457 -0.003419      19        1   1.680984
41     0.001457 -0.003419      20        1   1.680984
20     0.001458 -0.003419      21        1   1.680983
11     0.001456 -0.003419      22        1   1.680983
1      0.001459 -0.003418      23        1   1.680983
10     0.001455 -0.003419      24        1   1.680982
21     0.001458 -0.003418      25        1   1.680981
14     0.001455 -0.003419      26        1   1.680981
15     0.001455 -0.003419      27        1   1.680981
29     0.001455 -0.003419      28        1   1.680980
45     0.001458 -0.003419      29        1   1.680980
35     0.001456 -0.003419      30        1   1.680980
24     0.001458 -0.003419      31        1   1.680980
16     0.001458 -0.003419      32        1   1.680979
23     0.001459 -0.003418      33        1   1.680978
0      0.001455 -0.003419      34        1   1.680978
5      0.001457 -0.003419      35        1   1.680978
6      0.001456 -0.003419      36        1   1.680977
40     0.001456 -0.003419      37        1   1.680977
26     0.001458 -0.003419      38        1   1.680977
4      0.001456 -0.003419      39        1   1.680976
44     0.001457 -0.003419      40        1   1.680975
8      0.001455 -0.003419      41        1   1.680975
19     0.001456 -0.003419      42        1   1.680974
17     0.001459 -0.003418      43        1   1.680974
36     0.001456 -0.003419      44        1   1.680973
46     0.001458 -0.003419      45        1   1.680973
47     0.001456 -0.003419      46        1   1.680973
43     0.001456 -0.003419      47        1   1.680973
27     0.001458 -0.003419      48        1   1.680972
2      0.001456 -0.003419      49        1   1.680972
34     0.001455 -0.003419      50        1   1.680971, topic_info=          Term       Freq      Total Category  logprob  loglift
6          der  81.000000  81.000000  Default  21.0000  21.0000
7     internet  46.000000  46.000000  Default  20.0000  20.0000
1         kein  42.000000  42.000000  Default  19.0000  19.0000
5        nicht  47.000000  47.000000  Default  18.0000  18.0000
14     storung  44.000000  44.000000  Default  17.0000  17.0000
..         ...        ...        ...      ...      ...      ...
736   anschluß   0.040109   2.155021  Topic50  -7.5101   0.1018
104   leistung   0.039724   2.149903  Topic50  -7.5198   0.0946
213  glasfaser   0.040317   2.396373  Topic50  -7.5050   0.0008
318  telefonie   0.040728   4.113993  Topic50  -7.4948  -0.5294
274        weg   0.039819   2.729532  Topic50  -7.5174  -0.1417

[2412 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       1  0.514672        aber
115       2  0.171557        aber
84        4  0.741898   anschluss
178       2  0.520422        auch
108       3  0.328413     ausfall
...     ...       ...         ...
107       2  0.155721      wieder
16        2  0.893947        wlan
282

##### April

In [27]:
custom_lda.add_corpus(token_data["apr"], "apr")
custom_lda.visualize("apr")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27     0.013279  0.297131       1        1  12.861987
9      0.447251  0.198029       2        1   8.749994
28    -0.235641  0.248813       3        1   6.861373
17    -0.248648  0.413416       4        1   4.022848
8      0.314051 -0.199238       5        1   3.926969
26     0.401049 -0.007228       6        1   3.659346
22     0.228686  0.212443       7        1   3.556740
2     -0.187175  0.143458       8        1   3.539788
18    -0.252912 -0.012349       9        1   3.511950
23     0.372070  0.031164      10        1   3.448949
16    -0.097630  0.398568      11        1   3.428316
29     0.141401  0.315845      12        1   3.378090
13    -0.315898 -0.082686      13        1   3.238138
7     -0.120336  0.269243      14        1   3.089153
11     0.310745 -0.098756      15        1   2.774051
14    -0.420493 -0.080290      16        1   2.711540
20     0.154285 -0.221727      17        1   2.599353
24    -0.357208  0.190128      18        1   2.547695
6      0.148990  0.063932      19        1   2.423810
21     0.244760  0.319891      20        1   2.363229
15    -0.010302  0.122760      21        1   2.242999
19    -0.015436 -0.340522      22        1   2.201401
25     0.034134 -0.054764      23        1   2.192700
4      0.018384 -0.198646      24        1   2.076880
0     -0.373494 -0.265542      25        1   1.626834
5      0.068310 -0.454726      26        1   1.547857
1      0.266656 -0.339206      27        1   1.497481
3     -0.126536 -0.119108      28        1   1.385581
10    -0.146300 -0.419570      29        1   1.382579
12    -0.256039 -0.330463      30        1   1.152370, topic_info=                 Term        Freq       Total Category  logprob  loglift
1                kein  201.000000  201.000000  Default  21.0000  21.0000
14            storung  123.000000  123.000000  Default  20.0000  20.0000
7            internet  207.000000  207.000000  Default  19.0000  19.0000
5               nicht  186.000000  186.000000  Default  18.0000  18.0000
134         techniker  106.000000  106.000000  Default  17.0000  17.0000
...               ...         ...         ...      ...      ...      ...
1275          browser    0.099322    0.376177  Topic30  -6.2258   3.1317
154      unterbrechen    0.959436    8.568412  Topic30  -3.9578   2.2739
1                kein   11.732308  201.209601  Topic30  -1.4541   1.6213
392   leitungsproblem    0.003848    0.173155  Topic30  -9.4767   0.6566
0                 dsl    0.003848   27.216594  Topic30  -9.4767  -4.4008

[899 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
198       3  0.915820         ab
1062     19  1.125492  abbrechen
575      22  0.853195    abbruch
122      25  0.914475   abbruche
163       7  0.678549      abend
...     ...       ...        ...
161       3  0.948970    zuhause
534      22  1.024459     zuruck
338      20  0.685918   zusammen
196       1  0.592143       zwar
196      22  0.296072       zwar

[824 rows x 3 columns], R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 10, 29, 18, 9, 27, 23, 3, 19, 24, 17, 30, 14, 8, 12, 15, 21, 25, 7, 22, 16, 20, 26, 5, 1, 6, 2, 4, 11, 13])

In [26]:
# # new corpus
# corpus_apr = [custom_lda.id2word.doc2bow(doc) for doc in tokens_apr]
# predictions_apr = custom_lda.model[corpus_apr]

# # visual
# pyLDAvis.enable_notebook()
# visualization = pyLDAvis.gensim_models.prepare(custom_lda.model, corpus_apr, custom_lda.id2word, mds="mmds", R=21)
# visualization

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.122086 -0.295445       1        1  10.561252
32     0.141218  0.312399       2        1   6.003918
37    -0.336073  0.041759       3        1   3.840976
38     0.032688  0.056192       4        1   1.936764
42    -0.021641  0.026711       5        1   1.864730
9     -0.002380  0.008810       6        1   1.829273
43     0.001458 -0.003419       7        1   1.680994
0      0.001458 -0.003419       8        1   1.680990
18     0.001459 -0.003418       9        1   1.680989
23     0.001459 -0.003418      10        1   1.680988
13     0.001458 -0.003419      11        1   1.680985
20     0.001459 -0.003418      12        1   1.680984
6      0.001456 -0.003419      13        1   1.680984
10     0.001455 -0.003419      14        1   1.680983
1      0.001457 -0.003419      15        1   1.680983
46     0.001457 -0.003419      16        1   1.680983
17     0.001455 -0.003419      17        1   1.680982
31     0.001457 -0.003419      18        1   1.680982
36     0.001457 -0.003419      19        1   1.680982
25     0.001457 -0.003419      20        1   1.680981
47     0.001458 -0.003419      21        1   1.680981
22     0.001456 -0.003419      22        1   1.680981
30     0.001459 -0.003418      23        1   1.680981
45     0.001455 -0.003419      24        1   1.680980
34     0.001458 -0.003418      25        1   1.680980
14     0.001455 -0.003419      26        1   1.680979
40     0.001455 -0.003419      27        1   1.680979
33     0.001455 -0.003419      28        1   1.680979
49     0.001458 -0.003419      29        1   1.680978
15     0.001456 -0.003419      30        1   1.680978
21     0.001458 -0.003419      31        1   1.680978
39     0.001458 -0.003419      32        1   1.680978
29     0.001459 -0.003418      33        1   1.680978
27     0.001455 -0.003419      34        1   1.680978
28     0.001457 -0.003419      35        1   1.680978
11     0.001456 -0.003419      36        1   1.680977
8      0.001456 -0.003419      37        1   1.680977
4      0.001458 -0.003419      38        1   1.680977
26     0.001456 -0.003419      39        1   1.680976
44     0.001457 -0.003419      40        1   1.680976
5      0.001455 -0.003419      41        1   1.680976
41     0.001456 -0.003419      42        1   1.680976
12     0.001459 -0.003418      43        1   1.680975
19     0.001456 -0.003419      44        1   1.680975
48     0.001458 -0.003419      45        1   1.680974
24     0.001456 -0.003419      46        1   1.680974
35     0.001456 -0.003419      47        1   1.680973
3      0.001458 -0.003419      48        1   1.680972
16     0.001456 -0.003419      49        1   1.680968
2      0.001455 -0.003419      50        1   1.680966, topic_info=          Term       Freq      Total Category  logprob  loglift
6          der  81.000000  81.000000  Default  21.0000  21.0000
7     internet  46.000000  46.000000  Default  20.0000  20.0000
1         kein  42.000000  42.000000  Default  19.0000  19.0000
5        nicht  47.000000  47.000000  Default  18.0000  18.0000
14     storung  44.000000  44.000000  Default  17.0000  17.0000
..         ...        ...        ...      ...      ...      ...
736   anschluß   0.040108   2.155021  Topic50  -7.5101   0.1018
104   leistung   0.039722   2.149905  Topic50  -7.5198   0.0945
213  glasfaser   0.040317   2.396380  Topic50  -7.5050   0.0008
318  telefonie   0.040727   4.113961  Topic50  -7.4948  -0.5294
274        weg   0.039819   2.729541  Topic50  -7.5174  -0.1418

[2412 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       1  0.514667        aber
115       2  0.171556        aber
84        4  0.741898   anschluss
178       2  0.520425        auch
108       3  0.328415     ausfall
...     ...       ...         ...
107       2  0.155719      wieder
16        2  0.893958        wlan
282

##### May

In [28]:
custom_lda.add_corpus(token_data["may"], "may")
custom_lda.visualize("may")

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
27    -0.139096 -0.077547       1        1  15.397445
9      0.143964 -0.478289       2        1   9.200456
28     0.199892  0.310027       3        1   7.028282
8      0.325936 -0.182354       4        1   3.963611
13     0.318838  0.245350       5        1   3.922145
18     0.019122 -0.189288       6        1   3.886876
2     -0.002678  0.245541       7        1   3.694105
29     0.302341 -0.053811       8        1   3.534519
23     0.189622 -0.319511       9        1   3.507070
17    -0.430646  0.154328      10        1   3.448460
22     0.364119  0.098268      11        1   3.233433
16    -0.288790  0.154699      12        1   3.088938
11     0.030521 -0.324575      13        1   3.064982
14    -0.121207 -0.402115      14        1   3.002060
6      0.155428 -0.032005      15        1   2.864169
7     -0.102910  0.207843      16        1   2.824855
26     0.258388 -0.297153      17        1   2.570888
20    -0.230588 -0.173843      18        1   2.351188
25     0.004124 -0.023934      19        1   2.054877
15     0.131348  0.231155      20        1   2.006961
19     0.109860  0.402034      21        1   1.980552
1     -0.066586  0.415814      22        1   1.905828
24    -0.287444  0.225566      23        1   1.830016
21     0.415290 -0.003455      24        1   1.716037
4      0.184852  0.080794      25        1   1.604898
0     -0.396416 -0.215861      26        1   1.571309
12    -0.248000 -0.361808      27        1   1.329046
10    -0.432502 -0.062714      28        1   1.163952
3     -0.154228  0.036349      29        1   1.153172
5     -0.252553  0.390496      30        1   1.099869, topic_info=                  Term        Freq       Total Category  logprob  loglift
1                 kein  199.000000  199.000000  Default  21.0000  21.0000
7             internet  190.000000  190.000000  Default  20.0000  20.0000
134          techniker  118.000000  118.000000  Default  19.0000  19.0000
14             storung   99.000000   99.000000  Default  18.0000  18.0000
5                nicht  171.000000  171.000000  Default  17.0000  17.0000
..                 ...         ...         ...      ...      ...      ...
490  internetanschluss    1.095416    3.330729  Topic30  -3.7108   3.3979
36               hallo    2.707773   13.164229  Topic30  -2.8058   2.9286
511         herstellen    0.585425    2.046548  Topic30  -4.3373   3.2584
17             leitung    0.932893   18.886137  Topic30  -3.8714   1.5021
115               aber    0.666132   17.049138  Topic30  -4.2082   1.2676

[900 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
198       3  0.962351         ab
1062     15  1.035932  abbrechen
575      21  1.007674    abbruch
122      26  1.011493   abbruche
163      11  0.790805      abend
...     ...       ...        ...
161       3  0.992339    zuhause
534      21  1.208209     zuruck
338      24  0.969505   zusammen
196       1  0.584351       zwar
196      21  0.292176       zwar

[786 rows x 3 columns], R=21, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[28, 10, 29, 9, 14, 19, 3, 30, 24, 18, 23, 17, 12, 15, 7, 8, 27, 21, 26, 16, 20, 2, 25, 22, 5, 1, 13, 11, 4, 6])

In [29]:
# # new corpus
# corpus_may = [custom_lda.id2word.doc2bow(doc) for doc in tokens_may]
# predictions_may = custom_lda.model[corpus_may]

# # visual
# pyLDAvis.enable_notebook()
# visualization = pyLDAvis.gensim_models.prepare(custom_lda.model, corpus_may, custom_lda.id2word, mds="mmds", R=21)
# visualization 

### NMF

In [11]:
# Fit NMF model
nmf_model = gensim.models.Nmf(
    corpus = corpus,     # Document-Term Matrix
    id2word = id2word,   # Map word IDs to words
    num_topics = N_TOPICS,     # Number of latent topics to extract
    random_state = SEED,
    passes = 100,        # N° of passes through the corpus during training
    )

# Get the topics sorted by sparsity
pd.DataFrame(nmf_model.show_topics(), columns=["topic", "keywords"])

,topic,keywords
0,17,"0.365*""der"" + 0.015*""nicht"" + 0.013*""beheben"" ..."
1,18,"0.119*""techniker"" + 0.109*""zu"" + 0.066*""kommen..."
2,6,"0.289*""ein"" + 0.040*""techniker"" + 0.026*""haben..."
3,19,"0.314*""mein"" + 0.043*""nicht"" + 0.019*""auf"" + 0..."
4,11,"0.146*""nicht"" + 0.125*""da"" + 0.125*""warum"" + 0..."
5,14,"0.373*""nicht"" + 0.311*""funktionieren"" + 0.022*..."
6,3,"0.364*""festnetz"" + 0.056*""mein"" + 0.027*""mit"" ..."
7,16,"0.332*""gehen"" + 0.314*""nicht"" + 0.015*""telefon..."
8,1,"0.419*""wlan"" + 0.054*""verbindung"" + 0.015*""anz..."
9,7,"0.489*""telefon"" + 0.012*""beispielort"" + 0.012*..."


## Sklearn

In [12]:
vectorizer = TfidfVectorizer(stop_words=STOP_WORDS)

In [13]:
model_nmf = NMF(n_components=N_TOPICS, random_state=SEED)
model_lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=SEED)

### NMF

In [14]:
topic_pipeline = make_topic_pipeline(vectorizer, model_nmf)
topic_pipeline.fit(docs)
topicwizard.visualize(docs, model=topic_pipeline)

Preprocessing
Inferring topical content for documents.


### LDA

In [15]:
topic_pipeline = make_topic_pipeline(vectorizer, model_lda)
topic_pipeline.fit(docs)
topicwizard.visualize(docs, model=topic_pipeline)

Preprocessing
Inferring topical content for documents.
